# Airi
Autor: Gabriel Dornelles Monteiro, abril de 2022. Notebook nº3.5.

Construimos dois modelos nos notebooks anteriores, e como vimos, o processo para construir e treinar os estes modelos cresceu muito do primeiro para o segundo, se torna bastante impráticavel aumentarmos os modelos com o paradigma atual que utilizamos até agora. 

Para isso, iremos fazer o mesmo que os modernos frameworks de Machine Learning fazem, iremos modularizar todo nosso processo, isto é, criaremos módulos que executam o que precisamos, calculam o que deve ser calculado, para que dessa maneira, nosso trabalho seja encaixar estes módulos como encaixamos peças de Lego.

De maneira geral, cada um de nossos blocos terá o seguinte formato:

```py
class AiriLayer:
    def __init__(self):
       pass
  
    def __call__(self, x):
        self.forward(x)
    
    def forward(self, x):
        pass
    
    def backward(self, dout):
        pass
    
    def update(self, **kwargs):
        pass

    def zero_grad(self, **kwargs):
        pass
```
Onde :
- em  ```__init__``` temos um construtor padrão para nossa classe
- Utilizamos o magic method ```__call__``` para passar o método forward, ou seja, ao executarmos algo como:
```py
sample_module = AiriLayer()
sample_module(x)
```
Estaremos efetivamente, executando o método forward da classe.
- Em ```forward``` escreveremos o código que performa a computação de inferência do modelo
- Em ```backward``` escreveremos o código que calcula os gradientes para nosso bloco, e também que ira retornar o gradiente que deve seguir seu fluxo pelo modelo
- Em ```update``` escreveremos o código que irá realizar efetivamente todos os updates dos parâmetros pertencentes a nosso bloco.
- Em ```zero_grad``` iremos limpar a memória zerando todos os parâmetros que mantivemos em cache durante o processo de otimização.

----

Vejamos o exemplo de implementação para nosso bloco Linear, que é nossa matriz de pesos:

```py
class Linear(AiriLayer):

    def __init__(self, input_size=3072, hidden_size=10, reg=1e3, bias = True):
        self.config = None
        self.config_b = None
        self.bias = bias
        std = 1./ math.sqrt(input_size)
        self.w =  np.random.randn(input_size, hidden_size) * std
        self.b = np.zeros(hidden_size) if bias else None
        self.reg = reg
    
    def __call__(self, x):
        return self.forward(x)
    
    def forward(self, x, grad = True):
        if grad: 
            self.x = x
        return x@self.w + self.b
    
    def backward(self, dout):
        self.dW = self.x.T@dout
        self.dB = dout.sum(axis=0) if self.bias else None
        self.dW += self.reg * 2 * self.w 
        return dout@self.w.T 
        
    def update(self, lr=1e-3):
        self.w -= lr * self.dW
        self.b -= lr * self.dB
    
    def zero_grad(self):
        self.dW = None
        self.dB = None
        self.x = None
```

Veja que no construtor, inicializamos a matriz normalmente, como iremos treinar modelos maiores, utilizaremos a Kaiming Initialization. Perceba que simplesmente modularizamos o processo que construimos no Softmax Classifier, de maneira a deixar isto incrementável/escalável.

Para nossa layer Softmax:

```py
class Softmax(AiriLayer):

    def __init__(self, loss: str = "NLL"):
        self.loss_function = loss
        
    def __call__(self, x):
        return self.forward(x)

    def forward(self, scores, grad = True):
        self.batch_size = scores.shape[0]
        scores -= np.max(scores, axis=1, keepdims=True)
        scores_exp = np.exp(scores)
        softmax_matrix = scores_exp / np.sum(scores_exp, axis=1, keepdims=True) 
        if grad:
            self.softmax_matrix = softmax_matrix
        return softmax_matrix
    
    def NLLloss(self, y):
        loss = np.sum(-np.log(self.softmax_matrix[np.arange(self.batch_size), y]))
        loss /= self.batch_size
        return loss
    
    def backward(self, y):
        if self.loss_function == "NLL":
            self.softmax_matrix[np.arange(self.batch_size) ,y] -= 1
            self.softmax_matrix /= self.batch_size
            return self.softmax_matrix
        raise NotImplementedError("Unsupported Loss Function")
    
    def zero_grad(self):
        self.softmax_matrix = None
```

Novamente, não há nada de novo, o código aqui presente é copiado e colado do código que trabalhamos anteriormente.


----
O treinamento do Softmax Classifier agora é reduzido a:
```py
from layers import Linear, softmax

model = [
    Linear(input_size=3072, hidden_size=10, reg=1e3, bias=True),
    Softmax()
    ]

n_epoches = 500
x = train_images
y = train_targets

for i in range(n_epoches):
    
    for layer in model:
        x = layer.forward(x) # Não é necessario chamar o método forward. layer() é suficiente
    
    loss = model[-1].NLLloss(y)
    dout = model[-1].backward(y)
    
    for layer in reversed(model[:-1]):
        dout = layer.backward(dout)
        layer.update()
        layer.zero_grad()
```

Da mesma maneira, se quisermos agora treinar nossa Two Layer Neural Net, basta cria-la com blocos:

```py
model = [
    Linear(input_size=3072, hidden_size=100, reg=1e3, bias=True),
    Relu()
    Linear(input_size=100, hidden_size=10, reg=1e3, bias=True),
    Softmax()
]
```

E basta treinar da mesma maneira.


# Convoluções

O processo de convolução já foi amplamente utilizado em processamento de imagem, como filtros de detecção de borda (Sobel, Canny), filtros para borrar a imagem, dentre muitos outros. De maneira simples, o processo de convolução é dado por analisar uma pequena região da imagem repetidas vezes:

![full_padding_no_strides_transposed](https://user-images.githubusercontent.com/56324869/118333250-c1fa9c00-b4e1-11eb-99ac-d221e285976a.gif)

Onde criamos uma matriz quadrada (normalmente, mas retangulares também são usadas) e essa possui valores arbitŕarios, de maneira que conseguimos certos resultados com diferentes valores nesta matriz.

O exemplo base é o filtro sobel, dado pelas matrizes:

![image](https://user-images.githubusercontent.com/56324869/118333675-7e546200-b4e2-11eb-9cbc-32a0300dbb98.png)

![image](https://user-images.githubusercontent.com/56324869/118334477-1272f900-b4e4-11eb-9dfb-2d7d2e2e2173.png)

Que calcula as distâncias entre as cores, de maneira vertical e horizontal (cada matriz faz uma direção), dessa maneira, temos como resultado 0 quando não há diferença de cor na regiao analisada, e 1 (ou nesse caso 255) onde há uma diferença extrema de cor. Por isso, temos o filtro sobel como um detector de bordas.

![image](https://user-images.githubusercontent.com/56324869/163439169-d92e254c-7e38-4d55-b7b0-09081396017a.png)

Em nossos modelos, vamos utilizar os valores dentro dessa matriz como parâmetros que serão aprendidos pelo modelo, sendo então sua função aprender filtros que sejam relevantes para mostrar as caracterísisticas das imagens. Para isso, em cada camada de convolução, temos N número de filtros MxM, como por exemplo, 32 filtros 3x3, e o resultado dessas imagens é que iremos passar para nossas camadas lineares. 

Você pode ver uma implementação do código para o forward de uma camada de convolução nas layers de nossa lib, e também o seu backward. Como esta é uma operação mais custosa, a implementação utiliza Cython e necessita de um build. A mesma foi retirada do curso cs231n de Stanford, e re-implementada de maneira modular dentro de nossa lib.



# Implementando nossa ConvNet

O modelo que iremos rodar se encontra no script train.py dentro da lib airi. 

Modelo:

```py
    self.model = [
            Conv2D(in_channels=3, num_filters=16,filter_size=5, stride=1, pad=0),
            Relu(),
            Conv2D(in_channels=16, num_filters=16, filter_size=5, stride=1, pad=0),
            Relu(),
            Flatten(),
            Linear(input_size=9216, hidden_size=120, reg=self.reg),
            Relu(),
            Linear(input_size=120, hidden_size=84, reg=self.reg),
            Relu(),
            Linear(input_size=84, hidden_size=10, reg=self.reg),
            Softmax()
        ]
```

Iremos treinar o modelo por 20 épocas, com um batch size de 128, porém não mais e maneira aleatória. Dessa vez faremos o processo correto, que é:

- Iteramos o dataset inteiro em batches de 128, cada vez que passamos 128 exemplos, calculamos os gradientes e aplicamos o update. 
- Fazemos isso para todo o dataset 20 vezes.

Fazendo isso, verá que nosso algoritmo para otimização é extremamente fraco. Dado que implementamos tudo em python, nossa execução é muito lenta, e levamos cerca de 3 minutos e meio por época para treinar nosso modelo (na minha máquina). O nosso SGD é muito simples e pouco elaborado, por isso vamos ver um meio de otimizar nosso modelo de maneira mais eficiente

---

# Otimizadores

O SGD é bom, mas é muito lento, adicionaremos momentum, o que dará aceleração para ficar mais rápido quando encontrar uma boa direção em relação ao minimo da loss function.


## SGD + Momentum

Em particular, a perda pode ser interpretada como a altura de um terreno montanhoso (e, portanto, também para a energia potencial, pois U=mgh e, portanto, U∝h ). Inicializar os parâmetros com números aleatórios é equivalente a definir uma partícula com velocidade inicial zero em algum local deste terreno. O processo de otimização pode então ser visto como equivalente ao processo de simulação do vetor de parâmetros (ou seja, uma partícula) rolando na montanha.

Adicionar momentum ao SGD significa que estamos adicionando aceleração ao gradiente (análogo à física, onde f = ma, então o gradiente (negativo) é nesta visão, proporcional à aceleração da partícula). Temos uma interpretação física como uma bola rolando pela montanhosa função de perda, dessa vez adicionamos o atrito (coeficiente mu) e também a aceleração.

```py
## vanilla SGD ##
x += learning_rate * dx
########################

## SGD + Momentum ##
v = 0 # initialize velocity at 0
# momentum update
# mu usually something between 0.5 and 0.99
v = mu * v - learning_rate * dx # integrate velocity
x += v # integrate position
```

Normalmente, o momentum é aumentado em estágios posteriores de aprendizagem. Uma configuração típica é começar com momentum de cerca de 0,5 e gradualmente aumenta-lo para 0,99 ao longo das épocas.

## AdaGrad

Per parameter **Ada**ptive learning rate method for the **grad**ients. Este otimizador tem como objetivo trazer algo como um learning rate próprio para cada parâmetro do modelo, que é aumentado ou diminuido dinamicamente baseado nos últimos gradientes calculados pelo modelo. Para isso, ele adiciona o quadrado dos parâmetros em uma especie de cache:

```py
# AdaGrad update
cache += dx**2
x +=  - learning_rate * dx  / (np.sqrt(cache) + 1e-7)
```

Ao treinar por muito tempo, o update que fazemos se aproxima muito de zero, já que o cache é incrementado toda vez, eventualmente ele é grande demais, fazendo a operação do nosso update ser dividida por um valor muito alto, resultando em pouco update, e eventualmente nenhum update.

## RMSProp

Um otimizador baseado no treinamento em batches. Seu objetivo é corrigir o problema do AdaGrad, para isso, adiciona uma variavel **decay_rate**, que irá vazar um pouco do cache durante o processo de otimização, dessa maneira, o tempo para que se chegue a updates nulos é mais lento comparado ao AdaGrad.

```py
# RMSProp
cache = decay_rate * cache + (1 - decay_rate) * dx**2
x +=  - learning_rate * dx  / (np.sqrt(cache) + 1e-7)
```

## Adam
**Ada**ptative **M**oment estimation

O querido de todos, e amplamente utilizado na esmagadora maioria de modelos. Este otimizador é uma versão do RMSProp com a utilização de Momentum, e iremos utiliza-lo para treinar nossas ConvNets.

Sua ideia é algo como:

```py
# update first moment (momentum like), beta1 is like the friction we had in Momentum SGD.
m = beta1*m + (1-beta1)*dx 

# update second moment (RMSProp like), where we had a cache: 
v = beta2*v + (1-beta2)*(dx**2) 

# Adam update
x += - learning_rate * m / (np.sqrt(v) + 1e-7)
```


# Veja a diferença 

Veja como os diferentes otimizadores performam a descida de gradiente:

![3_-_NKsFHJb_-_Saddle_Point](https://user-images.githubusercontent.com/56324869/163443678-c1b4a5dc-3c02-4ca4-afce-856cccf99001.gif)

![1_STiRp7PW5yIrvYZupZA6nw](https://user-images.githubusercontent.com/56324869/163443917-d30dc066-1dfb-4c25-97c1-ecff45d01de5.gif)

Este é um campo com estudos ativos sempre, e verá que há outros como AdamW, dentre outros propostos:

![img](https://user-images.githubusercontent.com/56324869/163444019-803cfcca-5f21-442a-8aa0-e01cd83b6d1b.gif)

De maneira geral, Adam e RMSProp são amplamente utilizados hoje em dia tanto na academia quanto nos projetos reais.

# Treinamento

Para executar o treinamento da nossa ConvNet, basta executar o script train.py na lib airi. Iremos utilizar o módulo **pickle** do python para serializar nosso modelo em um arquivo binário e carrega-lo facilmente quando quiseremos (isto é exatamente o PyTorch faz com seus modelos!). 

Veremos os seus resultados no próximo notebook!